<a href="https://colab.research.google.com/github/brenoslivio/SME0828_DataScience/blob/master/Projects/3%20-%20Classifica%C3%A7%C3%A3o/Projeto3_Classificacao_B.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# SME0828 - Introdução à Ciência de Dados
# Projeto 3: Classificação 

### Alunos

Aline Fernanda da Conceição, Nº USP: 9437275

Breno Lívio Silva de Almeida, Nº USP: 10276675

Matheus Victal Cerqueira, Nº USP: 10276661

***Universidade de São Paulo, São Carlos, Brasil.***

## Exercícios sugeridos para a atividade avaliativa 3

Os seguintes exercícios foram sugeridos para entrega em 03 de novembro de 2020 para a disciplina SME0828 - Ciência de Dados oferecida pelo Instituto de Ciências Matemáticas e de Computação (ICMC) da Universidade de São Paulo (USP), a qual foi ministrada pelo professor Francisco Aparecido Rodrigues. Vamos trabalhar com o conjunto de dados [*Star dataset to predict star types*](https://www.kaggle.com/deepu1109/star-dataset). O dataset tem várias informações sobre características de estrelas como temperatura absoluta, luminosidade relativa, tipos de estrelas, entre outros. Vamos classificar os dados considerando vários classificadores conhecidos.



---



## Sumário

1. Desenvolvimento

  1.1 [Exercício 1](#ex1)

  1.2 [Exercício 2](#ex2)

  1.3 [Exercício 3](#ex3)

  1.4 [Exercício 4](#ex4)

  1.5 [Exercício 5](#ex5)

  1.6 [Exercício 6](#ex6)



---



## Desenvolvimento



### <a name="ex1"></a> 1 - Considere a base de dados sobre doenças cardíacas:https://www.kaggle.com/ronitf/heart-disease-uci 
### Faça o pré-processamento dos dados e classifique os pacientes de acordo com a variável “target”.  Considere os classificadores: Bayesiano paramétrico, Bayesiano não-paramétrico e Naive Bayes.

Foi considerada a base de dados de estrelas comentadas antes. Vamos trabalhar com a classificação para a variável Star color.

In [194]:
import numpy as np
import matplotlib.mlab as mlab
import matplotlib.pyplot as plt
import pandas as pd
import statistics

dataStar = pd.read_csv('https://raw.githubusercontent.com/brenoslivio/SME0828_DataScience/master/Projects/data/starsDataset.csv', 
        dtype={
          "Temperature (K)": np.int64,
          "Luminosity(L/Lo)": np.float64,
          "Radius(R/Ro)": np.float64,
          "Absolute magnitude(Mv)": np.float64,
          "Star type": np.int32,
          "Star color": str,
          "Spectral Class": str
        },
        na_values="",)

dataStar.replace("", np.nan, inplace=True)

dataStar.dropna(inplace=True)

# armazena os nomes das classes
classes = np.array(pd.unique(dataStar[dataStar.columns[-1]]), dtype=str)  

attributes = list(dataStar.columns)

dataStar.sample(5)

,Temperature (K),Luminosity(L/Lo),Radius(R/Ro),Absolute magnitude(Mv),Star type,Star color,Spectral Class
89,19860,0.0011,0.01310,11.34,2,Blue,B
215,32460,173800.0000,6.23700,-4.36,3,Blue,O
142,18290,0.0013,0.00934,12.78,2,Blue,B
11,3129,0.0122,0.37610,11.79,1,Red,M
100,33300,240000.0000,12.00000,-6.50,4,Blue,B


As classes para a cor das estrelas

In [195]:
classes

array(['M', 'B', 'A', 'F', 'O', 'K', 'G'], dtype='<U1')

In [196]:
data = dataStar.to_numpy()
data

array([[3068, 0.0024, 0.17, ..., 0, 'Red', 'M'],
       [3042, 0.0005, 0.1542, ..., 0, 'Red', 'M'],
       [2600, 0.0003, 0.102, ..., 0, 'Red', 'M'],
       ...,
       [8829, 537493.0, 1423.0, ..., 5, 'White', 'A'],
       [9235, 404940.0, 1112.0, ..., 5, 'White', 'A'],
       [37882, 294903.0, 1783.0, ..., 5, 'Blue', 'O']], dtype=object)

In [202]:
nrow,ncol = data.shape
y = data[:,-1]
X = data[:,0:ncol-2]

Normalizando os dados

In [203]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler().fit(X)
X = scaler.transform(X)

print('Dados transformados:')
print('Media: ', np.mean(X, axis = 0))
print('Desvio Padrao:', np.std(X, axis = 0))

Dados transformados:
Media:  [-1.48029737e-17  2.96059473e-17 -1.57281595e-17 -3.51570624e-17
 -2.40548322e-17]
Desvio Padrao: [1. 1. 1. 1. 1.]


In [185]:
np.isinf(X).any()

False

In [210]:
from sklearn.model_selection import train_test_split
p = 0.8 # fracao de elementos no conjunto de treinamento
x_train, x_test, y_train, y_test = train_test_split(X, y, train_size = p, random_state = 42)

In [211]:
from scipy.stats import multivariate_normal

#matrix to store the probabilities
P = pd.DataFrame(data = np.zeros((x_test.shape[0], len(classes))), columns = classes) 

Pc = np.zeros(len(classes)) #fraction of elements in each class
for i in np.arange(0, len(classes)):
    elements = tuple(np.where(y_train == classes[i]))
    Pc[i] = len(elements)/len(y_train)
    Z = x_train[elements,:][0]
    m = np.mean(Z, axis = 0)
    cv = np.cov(np.transpose(Z))
    print(elements, classes[i])
    for j in np.arange(0,x_test.shape[0]):
        x = x_test[j,:]
        pj = multivariate_normal.pdf(x, mean=m, cov=cv, allow_singular=True)
        P[classes[i]][j] = pj*Pc[i]


(array([  0,   1,   3,   5,   8,  10,  11,  13,  14,  15,  17,  20,  21,
        23,  25,  26,  28,  29,  33,  38,  41,  45,  46,  47,  48,  49,
        52,  58,  59,  60,  61,  62,  64,  74,  78,  80,  84,  85,  87,
        88,  89,  90,  91,  93,  94,  95,  96,  97,  99, 100, 103, 107,
       109, 110, 113, 115, 116, 118, 124, 125, 126, 132, 133, 134, 136,
       137, 139, 141, 142, 144, 145, 147, 148, 149, 150, 151, 155, 156,
       159, 163, 164, 165, 169, 170, 172, 179, 182, 185, 186, 188]),) M
(array([  2,   4,   7,  19,  22,  35,  36,  37,  51,  53,  63,  65,  66,
        73,  77,  79,  81,  86,  98, 102, 106, 117, 120, 128, 129, 130,
       131, 135, 160, 167, 171, 177, 178, 181, 184, 190]),) B
(array([ 12,  27,  34,  39,  43,  54,  57,  68,  70,  71,  92, 101, 108,
       111, 153, 158, 173]),) A
(array([  9,  24,  32,  40,  67,  72,  75,  82,  83, 104, 105, 122, 123,
       157, 166]),) F
(array([  6,  16,  18,  30,  31,  42,  44,  50,  55,  56,  69, 112, 114,
       119, 121

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:12: RuntimeWarning: Degrees of freedom <= 0 for slice
  if sys.path[0] == '':
/usr/local/lib/python3.6/dist-packages/numpy/lib/function_base.py:2455: RuntimeWarning: divide by zero encountered in true_divide
  c *= np.true_divide(1, fact)
/usr/local/lib/python3.6/dist-packages/numpy/lib/function_base.py:2455: RuntimeWarning: invalid value encountered in multiply
  c *= np.true_divide(1, fact)


ValueError: ignored

In [167]:
print(P)

              Red     Blue White  ...  Blue white   Blue-White
0   1.164425e-207   1.416466e-02  ...     0.005208    0.005208
1    1.578443e-02   6.936119e-06  ...     0.005208    0.005208
2    1.951768e-09   1.670173e-17  ...     0.005208    0.005208
3    0.000000e+00   1.923524e-49  ...     0.005208    0.005208
4    1.925578e-69   1.089449e-66  ...     0.005208    0.005208
5    2.738218e-20  7.042750e-100  ...     0.005208    0.005208
6    0.000000e+00   6.760165e-64  ...     0.005208    0.005208
7    5.337659e-95   1.200610e-02  ...     0.005208    0.005208
8    8.105019e-03   1.137317e-04  ...     0.005208    0.005208
9   3.586047e-152   1.446033e-02  ...     0.005208    0.005208
10   2.472444e-06   5.412012e-80  ...     0.005208    0.005208
11   0.000000e+00   2.746418e-03  ...     0.005208    0.005208
12   1.583533e-04  2.211404e-115  ...     0.005208    0.005208
13  4.774875e-231   5.055398e-62  ...     0.005208    0.005208
14   5.680405e-05  3.120247e-116  ...     0.005208    0

In [168]:
y_pred = []
#np.array(test_x.shape[0], dtype=str)
for i in np.arange(0, x_test.shape[0]):
    c = np.argmax(np.array(P.iloc[[i]]))
    y_pred.append(classes[c])
y_pred = np.array(y_pred, dtype=str)
print(y_pred)

['Yellowish' 'Yellowish' 'Yellowish' 'Yellowish' 'Yellowish' 'Yellowish'
 'Yellowish' 'Yellowish' 'Yellowish' 'Yellowish' 'Yellowish' 'Yellowish'
 'Yellowish' 'Yellowish' 'Yellowish' 'Yellowish' 'Yellowish' 'Yellowish'
 'Yellowish' 'Yellowish' 'Yellowish' 'Yellowish' 'Yellowish' 'Yellowish'
 'Yellowish' 'Yellowish' 'Yellowish' 'Yellowish' 'Yellowish' 'Yellowish'
 'Yellowish' 'Yellowish' 'Yellowish' 'Yellowish' 'Yellowish' 'Yellowish'
 'Yellowish' 'Yellowish' 'Yellowish' 'Yellowish' 'Yellowish' 'Yellowish'
 'Yellowish' 'Yellowish' 'Yellowish' 'Yellowish' 'Yellowish' 'Yellowish']


In [169]:
from sklearn.metrics import accuracy_score
score = accuracy_score(y_pred, y_test)
print('Accuracy:', score)

Accuracy: 0.020833333333333332


### <a name="ex2"></a> 2 - No classificar não-paramétrico, verifique o efeito do hiperparâmetro h na classificação dos dados de diabetes, encontrando seu melhor valor: https://www.kaggle.com/uciml/pima-indians-diabetes-database
